In [34]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score
import numpy as np
from gensim.models import FastText
from zemberek import TurkishMorphology

In [35]:
path_csv_file = "latest_hukums_with_classes_csv_file1.csv"
pos_tagger = TurkishMorphology.create_with_defaults()
df = pd.read_csv(path_csv_file)
df.head()

2022-05-19 13:02:56,903 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 5.886160612106323



,Unnamed: 0.1,Unnamed: 0,ictihat,Suç,YeniSuclar
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar


In [36]:
label_map = {}
count = 0
for label in df.YeniSuclar.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.YeniSuclar.map(label_map)
#print(label_map)
df.head()

,Unnamed: 0.1,Unnamed: 0,ictihat,Suç,YeniSuclar,NUM_LABEL
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar,0
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar,1
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar,2
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar,1
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar,0


In [37]:
df = df.dropna(how='any',axis=0)

In [38]:
df.isnull().sum()

Unnamed: 0.1    0
Unnamed: 0      0
ictihat         0
Suç             0
YeniSuclar      0
NUM_LABEL       0
dtype: int64

In [43]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))
    just_words = [word for word in lower_case_only if word not in stopwords_tr]
    words_lemma = []
    for word in just_words:
        tags = pos_tagger.analyze(word)
        if len(tags.analysis_results) > 0:
            words_lemma.append(tags.analysis_results[0].item.lemma)
        else:
            words_lemma.append(word)
    return words_lemma

In [44]:
df.iloc[:, 2] = df.iloc[:, 2].apply(lambda x: " ".join(clean_text(x)))
df = df[['ictihat', 'NUM_LABEL']]
df

,ictihat,NUM_LABEL
0,hüküm hüküm açık geri temyiz istek ret yerel m...,0
1,hüküm hüküm açık geri temyiz istek ret yerel m...,1
2,hüküm hüküm açık geri temyiz istek ret yerel m...,2
3,hüküm beraat yerel mahkeme bozmak üzerine verm...,1
4,hüküm beraat yerel mahkeme bozmak üzerine verm...,0
...,...,...
19275,hüküm sanık suç işlemek amaç örgüt kurmak suç ...,14
19276,hüküm Bakırköy asliye ceza mahkeme esas karmak...,4
19277,hüküm mahkûmiyet yerel mahkeme vermek hüküm te...,0
19278,hüküm mahkûmiyet yerel mahkeme vermek hüküm te...,1


In [45]:
df_labeled = df.copy()
df_labeled['NUM_LABEL'] = '__label__' + df_labeled['NUM_LABEL'].astype(str)
df_labeled.head()

,ictihat,NUM_LABEL
0,hüküm hüküm açık geri temyiz istek ret yerel m...,__label__0
1,hüküm hüküm açık geri temyiz istek ret yerel m...,__label__1
2,hüküm hüküm açık geri temyiz istek ret yerel m...,__label__2
3,hüküm beraat yerel mahkeme bozmak üzerine verm...,__label__1
4,hüküm beraat yerel mahkeme bozmak üzerine verm...,__label__0


In [46]:
ictihat_label = df_labeled['NUM_LABEL'] + " " + df_labeled['ictihat']
ictihat_label

0        __label__0 hüküm hüküm açık geri temyiz istek ...
1        __label__1 hüküm hüküm açık geri temyiz istek ...
2        __label__2 hüküm hüküm açık geri temyiz istek ...
3        __label__1 hüküm beraat yerel mahkeme bozmak ü...
4        __label__0 hüküm beraat yerel mahkeme bozmak ü...
                               ...                        
19275    __label__14 hüküm sanık suç işlemek amaç örgüt...
19276    __label__4 hüküm Bakırköy asliye ceza mahkeme ...
19277    __label__0 hüküm mahkûmiyet yerel mahkeme verm...
19278    __label__1 hüküm mahkûmiyet yerel mahkeme verm...
19279    __label__2 hüküm mahkûmiyet yerel mahkeme verm...
Length: 19269, dtype: object

In [47]:
df.to_csv("datas-zemberek.csv", index=False)

In [48]:
print(label_map)

{'Şerefe karşı suçlar': 0, 'Hürriyete karşı suçlar': 1, 'Vücut dokunulmazlığına karşı suçlar': 2, 'others': 3, 'Mal varlığına karşı suçlar': 4, 'Kamu idaresinin güvenilirliğine ve işleyişine karşı suçlar': 5, 'Kamunun sağlığına karşı suçlar': 6, 'Anayasal düzene ve bu düzenin işleyişine karşı suçlar': 7, 'Adliyeye karşı suçlar': 8, 'Hayata karşı suçlar': 9, 'Cinsel dokunulmazlığa karşı suçlar': 10, 'Ekonomi, sanayi ve ticarete ilişkin suçlar': 11, 'Bilişim alanında suçlar': 12, 'Aile düzenine karşı suçlar': 13, 'Kamu güvenine karşı suçlar': 14, 'Çevreye karşı suçlar': 15, 'Genel ahlaka karşı suçlar': 16, 'Göçmen kaçakçılığı ve insan ticareti': 17, 'Genel tehlike yaratan suçlar': 18}
